# Notebook for analysing data to find potential signals within
# 

### How does this script work?
First saved data in .txt form is loaded in and the dataset is extracted. The dataset is then ran through a pipeline looking for potential signals within the data and finally Dataframes are created with the characteristics of the full dataset and any signals within
# 
### How does the pipeline work?
The data is first analysed my a machine learning model. This model is un-supervised and is looking for regions in the data where there is high volatility, indicating a potential region of interest where a signal may be.

The predictions made by the model are then looked at and any regions of interest are saved and looked into with more detail. This invloves using a cross correlation function to determine if the signal has the characteristic curve, if the extracted signal is extremely long then the cross correlation function is used to see if there is a signal within the region of interest. If the cross correlation functions show a signal is present then they are analysed looking at the length, number of peaks and height of the peaks as well as the depth of the dip. If the cross correlation function shows no signal is present then the region of interest is discarded.

In [ ]:
import MLDetectionFunctionsPythonV9_5_NoRestricts_MLcreate as EEGanalysis
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, Output, fixed, interact_manual
from IPython.display import display, clear_output
import numpy as np
import math
import mat73
import pickle

In [ ]:
#I think need to add some length to end of extraction so can be cropped down





#AMAZING IDEA

#What if I saved all individual areas of interest extracted by the model
#this includes seizure events and non-seizure events
#use this to train a model that can classify each extracted area of interest

#save both raw and smoothed AoI and see which has better results



#Try first - just key infor in dataframe to train with

#using the extracted dataframe at the end of the code use this with either 'Signal' or 'Noise'
#to train a model which can then be used to classify extracted AoI instead of full signals

#trim down dataframe so doesnt include confusing info eg start and end times
#intead just have signal length

#include cross-corr score as well in dataframe
#think of other things to input into the dataframe

#maybe use rounded signal length (in seconds) as well as rounded dip depth, medium peak height

#look at peak density throughout signal
#rolling average over 10secs and pick median, mean or peak value?





#this looks to be a very good idea


#TO CHANGE
#If subsignal found i.e. 12.1, 12.2 (from 12.0) then need to delete the 12.0 signal

# Importing data to analyse

### Each dataset should be saved as a .txt file

# Choosing which dataset to analyse

In [ ]:
datasetusing = datamay36

#check 5, boarderline on signal/noise (especially 3rd signal)

#8 is good for testing with 'clean' dataset that works v well with original 9_5 version

#11 has detection issues
#14 is cool as works well
#15 cool too
#16 is v good
#17 detection issues due to dip depth too small and thus removing signal


In [ ]:
realdatausing = EEGanalysis.getdata(datasetusing)
realdatausing

In [ ]:
for i in range(4):
    print(i*(0.25*len(realdatausing)))
    print(int(i*(0.25*len(realdatausing))))
    print((i+1)*(0.25*len(realdatausing)))
    print(int((i+1)*(0.25*len(realdatausing))))
    print()
    locals()["filesub"+str(i+1)] = realdatausing[int(i*(0.25*len(realdatausing))):int((i+1)*(0.25*len(realdatausing)))]


In [ ]:
filesub1

In [ ]:
realdatausing = filesub1

In [ ]:
DFnorm, signal = EEGanalysis.CreateDataframe(realdatausing)
DFnormres, signalres2 = EEGanalysis.CreateDataframe(realdatausing)

# Formatting the curve data to use in cross correlation to determine validity of potential signals

In [ ]:
curvedata = EEGanalysis.getdata(curvedata)

In [ ]:
timecurve, signalcurve = EEGanalysis.timesignal(curvedata)

In [ ]:
signalcurveave = EEGanalysis.movingaverage(signalcurve[:,1],10000)

# Predicting the locations of potential signals within the dataset

In [ ]:
StartVals, EndVals, signalCURVES = EEGanalysis.PredictSignal(DFnorm, signal,350000)

# Analysing the whole dataset before and potential signals have been removed

In [ ]:
MainSignalDFtestfull = EEGanalysis.AnalyseSignalFullData(signal)

# Extracting potential individual signals, determining their validity and analysing them where necessary

In [ ]:
MainSignalDF, MainSignalDFraw, MainSignalDFccextracts,redsig = EEGanalysis.AnalyseSignalsFailsafeVer(StartVals,EndVals,signal,signalres2,signalCURVES,DFnorm,signalcurveave,250000)


# Analysing the remaining data after valid signals have been removed

In [ ]:
MainSignalDFtestfullthird = EEGanalysis.AnalyseSignalFullData(redsig)

# Dataframe containing characteristics of full original data

In [ ]:
MainSignalDFtestfull

# Dataframe containing characteristics of initially extracted signals, not including any second tier extractions

In [ ]:
MainSignalDFraw

# Dataframe containing characteristics of only second tier extractions

### This is where a signal was within an initial extraction but the original signal contained more data then just the signal

### The signal is extracted using cross correlation to trim out the 'excess' data from the original extraction

In [ ]:
MainSignalDFccextracts

# Dataframe containing the characteristics of the full data with signals removed

In [ ]:
MainSignalDFtestfullthird

# Combining the dataframes containing original extracted signals and second tier extractions

### Original extractions are labelled with .0 i.e. 5.0 or 8.0

### Second tier extractions are labelled with .1 .2 .3 depending on how many second tier extractions there are within one original signal  i.e original signal 6 may have two signals within it and they would be labelled 6.1 and 6.2   original signal 9 may have one signal within it and would be labelled 9.1

In [ ]:
MainSignalDFtot = MainSignalDFraw.append(MainSignalDFccextracts, ignore_index=True)


In [ ]:
MainSignalDFtot['SignalNumber'] = MainSignalDFtot['SignalNumber'].str.split('Signal').str.join('')

MainSignalDFtot['SignalNumber'] = pd.to_numeric(MainSignalDFtot['SignalNumber'])

MainSignalDFtot=MainSignalDFtot.sort_values('SignalNumber')


In [ ]:
MainSignalDF = MainSignalDF.rename(columns={"Startsig": "Start Time (s)", "Endsig": "End Time (s)"
                                           , "SignalLength": "Signal Length (s)"
                                           , "NoPeaksBeforeGlobal": "Peaks Before Global Min"
                                           , "NoPeaksAfterGlobal": "Peaks After Global Min"
                                           , "NoDipsBeforeGlobal": "Dips Before Global Min"
                                           , "NoDipsAfterGlobal": "Dips After Global Min"
                                           , "DipDepth": "Global Min Depth"
                                           , "Start": "Start Index"
                                           , "End": "End Index"})

In [ ]:
MainSignalDF

In [ ]:
#MainSignalDF['SignalNumber'] = [1,2,3,4,5]
#MainSignalDF

In [ ]:
#MainSignalDF.to_csv('Dataset7Output.csv', index = False)

In [ ]:
realdataV2 = EEGanalysis.getdata(datasetusing)
realdataV2

In [ ]:
for i in range(4):
    print(i*(0.25*len(realdataV2)))
    print(int(i*(0.25*len(realdataV2))))
    print((i+1)*(0.25*len(realdataV2)))
    print(int((i+1)*(0.25*len(realdataV2))))
    print()
    locals()["filesubV2"+str(i+1)] = realdataV2[int(i*(0.25*len(realdataV2))):int((i+1)*(0.25*len(realdataV2)))]


In [ ]:
filesubV21

In [ ]:
realdataV2 = filesubV21

In [ ]:
DFnorm3, signalres3 = EEGanalysis.CreateDataframe(realdataV2)

In [ ]:
MainSignalDF = MainSignalDF.fillna(0)
MainSignalDF

In [ ]:
MainSignalDF = MainSignalDF.sort_index()
MainSignalDF

In [ ]:
#MainSignalDF = MainSignalDF.drop([1])
#MainSignalDF

In [ ]:
MainSignalDFpred = MainSignalDF.sort_index()

In [ ]:
MainSignalDFpred = MainSignalDFpred.astype(float).round(0).astype(int)


In [ ]:
MainSignalDFpred = MainSignalDFpred.drop('Start Time (s)', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('End Time (s)', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('Start Index', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('End Index', axis=1)
MainSignalDFpred = MainSignalDFpred.drop('SignalNumber', axis=1)
MainSignalDFpred

In [ ]:
MLclassifier = pickle.load(open('PotentialSignalClassifier.sav', 'rb'))

In [ ]:
MainSignalDFpredar = MainSignalDFpred.to_numpy()

In [ ]:
prediction = MLclassifier.predict(MainSignalDFpredar)

In [ ]:
prediction

In [ ]:
print('All Curves Overlaid:')
for a in range(len(MainSignalDF)):
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])

In [ ]:
print('All Curves Staggared:')
for a in range(len(MainSignalDF)):
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1]-2000*(a+1))

In [ ]:
for a in range(len(MainSignalDF)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%MainSignalDFpred['Signal Length (s)'][a])
    print(prediction[a])
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    plt.show()
    print()
    print()

In [ ]:
for a in range(len(MainSignalDF)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%MainSignalDFpred['Signal Length (s)'][a])
    print('Prediction:%s' %prediction[a])
    plt.plot(signalres3[MainSignalDF['Start Time (s)'][a]*5000:MainSignalDF['End Time (s)'][a]*5000,1])
    plt.show()
    
    def f(Signal):
            1+1

    #locals()["my_result"+str(a)] = interactive(f, Signal=True)
    locals()["my_result"+str(a)] = widgets.RadioButtons(options=['Signal', 'Noise'],description='Signal or Noise:',disabled=False)

    display(locals()["my_result"+str(a)])
    
    locals()["UnuseableWhy"+str(a)] = input("If Signal Unuseable, Why? (Leave Blank and Press Enter if Useable)")
    
    
    
    input("Press Enter to Confirm")
    

    print()
    print()
    clear_output()

In [ ]:
for i in range(len(MainSignalDF)):
    print(locals()["my_result"+str(i)].value)

In [ ]:
MainSignalDF['User Entry'] = np.nan
MainSignalDF['Unuseable Reason'] = np.nan

for i in range(len(MainSignalDF)):
    MainSignalDF['User Entry'][i] = locals()["my_result"+str(i)].value
        
for i in range(len(MainSignalDF)):
        MainSignalDF['Unuseable Reason'][i] = locals()["UnuseableWhy"+str(i)]

In [ ]:
MainSignalDF

In [ ]:
for a in range(len(MainSignalDF)):
    if MainSignalDF['User Entry'][a] == 'Noise':
        MainSignalDF = MainSignalDF.drop([a])
        
MainSignalDF = MainSignalDF.reset_index()

In [ ]:
MainSignalDF

In [ ]:
globals()[iter] = 0

In [ ]:
DropdownOptions = []
for i in range(len(MainSignalDF)):
    DropdownOptions.append(('Signal %s'%str(i+1), i))

# Run the following cell two times, and interact with it the on the second run

In [ ]:
%matplotlib notebook
from numpy import pi, sin
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons


SignalSelector = widgets.Dropdown(
    options=DropdownOptions,
    value=globals()[iter],
    description='Signal:',
)

def reset_button_on_clicked(mouse_event):
    start_slider.reset()
    end_slider.reset()
    
def confirm_button_on_clicked(mouse_event):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    highlightzone = ax.axvspan(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='green', alpha=0.25)
    fig.canvas.draw_idle()
    globals()["StartCrop"+str(globals()[iter])] = start_slider.val
    globals()["EndCrop"+str(globals()[iter])] = end_slider.val
    
    
def sliders_on_changedstart(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
def sliders_on_changedend(val):
    ax.cla()
    [line] = ax.plot(x, y, linewidth=0.5, color='black')
    highlightplotstart = ax.axvline(((start_slider.val/200)*(np.max(x)-np.min(x))) + np.min(x), color='green', zorder=-1)
    highlightplotend = ax.axvline(np.max(x) - ((end_slider.val/200)*(np.max(x)-np.min(x))), color='red', zorder=-1)
    fig.canvas.draw_idle()
    
    
x = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,0]

y = signalres3[MainSignalDF['Start Time (s)'][globals()[iter]]*5000:MainSignalDF['End Time (s)'][globals()[iter]]*5000,1]
    


axis_color = 'lightgoldenrodyellow'

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)

fig.subplots_adjust(left=0.25, bottom=0.25)

[line] = ax.plot(x, y, linewidth=0.5, color='black')



start_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03], facecolor=axis_color)
start_slider = Slider(start_slider_ax, 'Start', 0, 100, valinit=0)

end_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03], facecolor=axis_color)
end_slider = Slider(end_slider_ax, 'End', 0, 100, valinit=0)

#highlightplot = ax.axvspan(start_slider.val/2, 100 - end_slider.val/2, color='green', alpha=0.5)
highlightplotstart = ax.axvline(np.min(x), color='green', zorder=-1)
highlightplotend = ax.axvline(np.max(x), color='red', zorder=-1)
    
start_slider.on_changed(sliders_on_changedstart)
end_slider.on_changed(sliders_on_changedend)

reset_button_ax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
reset_button = Button(reset_button_ax, 'Reset', color='red', hovercolor='0.975')

confirm_button_ax = fig.add_axes([0.25, 0.025, 0.1, 0.04])
confirm_button = Button(confirm_button_ax, 'Confirm', color='green', hovercolor='0.975')
    
reset_button.on_clicked(reset_button_on_clicked)
confirm_button.on_clicked(confirm_button_on_clicked)

    
plt.show()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])
        globals()[iter] = change['new']
        
        %rerun
       

SignalSelector.observe(on_change)

display(SignalSelector)

In [ ]:
SignalCropPercs = pd.DataFrame(columns = ['Signal', 'Start Perc', 'End Perc'])
Signallist = []
Startlist = []
Endlist = []
for i in range(len(MainSignalDF)):
    Signallist.append('Signal %s'%str(i+1))
    Startlist.append(globals()["StartCrop"+str(i)])#SignalCropPercs['Start Perc'][i] = ['StartCrop%s'%i]
    Endlist.append(globals()["EndCrop"+str(i)])#SignalCropPercs['End Perc'][i] = ['EndCrop%s'%i]
    
SignalCropPercs['Signal'] = Signallist
SignalCropPercs['Start Perc'] = Startlist
SignalCropPercs['End Perc'] = Endlist

SignalCropPercs

In [ ]:
MainSignalDF

In [ ]:
StartIndice = MainSignalDF['Start Time (s)']*5000
EndIndice = MainSignalDF['End Time (s)']*5000
SignalLengthIndice = EndIndice - StartIndice
SignalMidpointIndice = SignalLengthIndice/2

In [ ]:
StartIndice

In [ ]:
EndIndice

In [ ]:
SignalLengthIndice/5000

In [ ]:
SignalMidpointIndice/5000

In [ ]:
CropStartIndice = StartIndice + (SignalCropPercs['Start Perc']/100)*SignalMidpointIndice
#100% = midpoint
#0% = startpoint

In [ ]:
CropEndIndice = EndIndice - (SignalCropPercs['End Perc']/100)*SignalMidpointIndice

In [ ]:
CropStartIndice

In [ ]:
CropEndIndice

In [ ]:
CropSignalTEST = pd.DataFrame(columns = ['Start', 'End'])
CropSignalTEST['Start'] = CropStartIndice/5000
CropSignalTEST['End'] = CropEndIndice/5000
CropSignalTEST

In [ ]:
%matplotlib inline
for a in range(len(CropSignalTEST)):
    print('Graph of Extracted Curve %s'%a)
    print('Signal Length: %ss'%(int(CropSignalTEST['End'][a])-int(CropSignalTEST['Start'][a])))
    plt.rcParams['figure.figsize'] = [100, 50]
    plt.plot(signalres3[int(CropSignalTEST['Start'][a]*5000):int(CropSignalTEST['End'][a]*5000),1])
    plt.show()
    print()
    print()

In [ ]:

MainSignalDFTEST = EEGanalysis.AnalyseSignalsCROPPED(CropSignalTEST['Start']*5000,CropSignalTEST['End']*5000,signalres3)

In [ ]:
MainSignalDFTEST

In [ ]:
print('All Curves Overlaid:')
for a in range(len(MainSignalDFTEST)):
        
    argmin = np.argmin(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,1])
    signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] = range(0,len(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0]))
    signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] = signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] - argmin
    
    plt.plot(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0],signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,1])

In [ ]:
print('All Curves Staggared:')
count = 0
offset = 4000
for a in range(len(MainSignalDFTEST)):
    
    argmin = np.argmin(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,1])
    signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] = range(0,len(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0]))
    signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] = signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0] - argmin
    
    plt.plot(signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,0],signalres3[MainSignalDFTEST['Startsig'][a]*5000:MainSignalDFTEST['Endsig'][a]*5000,1]-offset*(count+1))
    count = count+1

In [ ]:
drugStart = [0,5000]
drugEnd = [5000,7500]

drugsNumber = input("Number of Drugs Used? ")
drugName = list()
drugColour = list()
for i in range(int(drugsNumber)):
    locals()["Drug"+str(i)] = input("Name of Drug Number %s: " %str(i+1))
    locals()["DrugColour"+str(i)] = input("Colour of Drug Number %s: " %str(i+1))
    drugName.append(locals()["Drug"+str(i)])
    drugColour.append(locals()["DrugColour"+str(i)])
    
print(drugName) 
print(drugColour)

In [ ]:
#go through each row of MainSignalDFTEST

#if start time of signal is 

In [ ]:
MainSignalDFTEST['Startsig'][0]

In [ ]:
drugUsed = [None]*len(MainSignalDFTEST)
for i in range(len(MainSignalDFTEST)):    
    for x in range(len(drugStart)):
        if MainSignalDFTEST['Startsig'][i] - drugEnd[x] < 0:
            drugUsed[i] = drugName[x]
            break
            
MainSignalDFTEST['Drug'] = drugUsed

In [ ]:
MainSignalDFTEST

In [ ]:
testtest = MainSignalDFTEST.loc[(MainSignalDFTEST['Drug'] == 'B')]
testtest = testtest.reset_index()
testtest

In [ ]:
for i in range(len(drugName)):

    MainSignalDrugUsing = MainSignalDFTEST.loc[(MainSignalDFTEST['Drug'] == drugName[i])]
    MainSignalDrugUsing = MainSignalDrugUsing.reset_index()
    print('All %s Curves Overlaid:'%drugName[i])
    for a in range(len(MainSignalDrugUsing)):
        
        argmin = np.argmin(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1])
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = range(0,len(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0]))
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] - argmin
    
        plt.plot(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0],signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1])
    plt.show()
    
    
    
    print('All %s Curves Staggared:'%drugName[i])
    count = 0
    offset = 4000
    for a in range(len(MainSignalDrugUsing)):
    
        argmin = np.argmin(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1])
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = range(0,len(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0]))
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] - argmin
    
        plt.plot(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0],signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1]-offset*(count+1))
        count = count+1
    plt.show()
       
        
        
        
        
        
        

In [ ]:
import matplotlib.patches as mpatches
count = 0
for i in range(len(drugName)):

    MainSignalDrugUsing = MainSignalDFTEST.loc[(MainSignalDFTEST['Drug'] == drugName[i])]
    MainSignalDrugUsing = MainSignalDrugUsing.reset_index()
    print('All %s Curves Staggared:'%drugName[i])
    #count = 0
    offset = 4000
    drugoffset = 8000*int(i)
    for a in range(len(MainSignalDrugUsing)):
        
        argmin = np.argmin(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1])
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = range(0,len(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0]))
        signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] = signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0] - argmin
    
        plt.plot(signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,0],signalres3[MainSignalDrugUsing['Startsig'][a]*5000:MainSignalDrugUsing['Endsig'][a]*5000,1]-offset*(count+1)-drugoffset,c=drugColour[i], label=drugName[i])
        count = count+1
        
#plt.legend(fontsize=75)
leg = plt.legend(drugName, labelcolor=drugColour, fontsize=100, handlelength=0, handletextpad=0)
for item in leg.legendHandles:
    item.set_visible(False)
#plt.legend(handles=[mpatches.Patch(color=drugColour[0], label=drugName[0]),mpatches.Patch(color=drugColour[1], label=drugName[1]),mpatches.Patch(color=drugColour[2], label=drugName[2])], fontsize=75)
plt.show()

In [ ]:
Drug0

# GOT UP TO HERE